# 3.1 Data Preparation

In [ ]:
#setup path
path = 'C:/Users/mark/Documents/GitHub/honours/dev/package/'
import sys; sys.path.append(path)

#standard imports
import numpy as np
import pandas as pd
import datetime
from scipy import stats
from collections import OrderedDict

#third party imports
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import matplotlib.dates as md
from matplotlib import rcParams
import seaborn as sns

#local imports
import analysis.dataPreparer as dp
import analysis.getStages as gs

In [ ]:
#setup
%matplotlib inline
rcParams['figure.figsize'] = 20,6
sns.set_style("whitegrid")

def set_style():
    plt.style.use(['seaborn-white', 'seaborn-paper'])
    matplotlib.rc("font", family="Times New Roman")

In [ ]:
#constants
input_path = path+"analysis/input/master.db"
flatten_config = path+"analysis/config/master_feature.sql"
raw_flat_file = path+"analysis/output/temp/raw.csv"
clean_flat_file = path+"analysis/output/temp/clean.csv"
output_path = path+"analysis/output/temp/output.db"
#output_path = path+"analysis/output/autoVC/8/test/2015-04-06/2017-04-03/label_clean.csv"

dp.flatten_file(input_path, flatten_config, raw_flat_file, "feature")
dp.clean_file(raw_flat_file, clean_flat_file)
dp.load_file(output_path, clean_flat_file, "feature")
df = dp.export_dataframe(output_path, "feature")
df_backup = df.copy()

In [ ]:
#reload data from memory
df = df_backup

In [ ]:
df = df.sample(50000)

In [ ]:
stage_features = dict(
    Age = 'confidence_context_broader_company_age_number',
    FundingRounds = 'confidence_validation_funding_rounds_number',
    FundingRaised = 'confidence_validation_funding_raised_value_total_number',
    SeriesA = 'confidence_validation_funding_round_codes_list_a',
    SeriesB = 'confidence_validation_funding_round_codes_list_b',
    SeriesC = 'confidence_validation_funding_round_codes_list_c',
    SeriesD = 'confidence_validation_funding_round_codes_list_d',
    SeriesE = 'confidence_validation_funding_round_codes_list_e',
    SeriesF = 'confidence_validation_funding_round_codes_list_f',
    SeriesG = 'confidence_validation_funding_round_codes_list_g',
    SeriesH = 'confidence_validation_funding_round_codes_list_h',
    Closed = "keys_company_status_closed_bool",
    Acquired = "keys_company_status_acquired_bool",
    IPO = "keys_company_status_ipo_bool"
)

In [ ]:
stage_features = dict(
    Age = 'outcome_age_number',
    FundingRounds = 'outcome_funding_rounds_number',
    FundingRaised = 'outcome_funding_raised_value_total_number',
    SeriesA = 'outcome_funding_round_codes_list_a',
    SeriesB = 'outcome_funding_round_codes_list_b',
    SeriesC = 'outcome_funding_round_codes_list_c',
    SeriesD = 'outcome_funding_round_codes_list_d',
    SeriesE = 'outcome_funding_round_codes_list_e',
    SeriesF = 'outcome_funding_round_codes_list_f',
    SeriesG = 'outcome_funding_round_codes_list_g',
    SeriesH = 'outcome_funding_round_codes_list_h',
    Closed = "outcome_closed_bool",
    Acquired = "outcome_acquired_bool",
    IPO = "outcome_ipo_bool"
)

In [ ]:
def create_stages(df, **features):
    df2 = df.copy()
    df2["keys_company_stage"] = "Other"
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["Closed"]] >= 1), "Closed", df2["keys_company_stage"])
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["Acquired"]] >= 1), "Acquired", df2["keys_company_stage"])
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["IPO"]] >= 1), "IPO", df2["keys_company_stage"])
    df2["keys_company_stage_series-d+"] = df2[[features["SeriesD"],features["SeriesE"],features["SeriesF"],features["SeriesG"],features["SeriesH"]]].sum(axis=1)
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2["keys_company_stage_series-d+"] >= 1), "Series D+", df2["keys_company_stage"])
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["SeriesC"]] >= 1), "Series C", df2["keys_company_stage"])
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["SeriesB"]] >= 1), "Series B", df2["keys_company_stage"])
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["SeriesA"]] >= 1), "Series A", df2["keys_company_stage"])
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["FundingRaised"]] >= 0), "Seed", df2["keys_company_stage"])
    age_new_cutoff = df2[features["Age"]][df2["keys_company_stage"] == "Seed"].quantile(0.75)
    df2["keys_company_stage"] = np.where((df2["keys_company_stage"] == "Other") & (df2[features["Age"]] <= age_new_cutoff), "Pre-Seed", df2["keys_company_stage"])
    group_stages = {"Other" : "Excluded", "Closed" : "Excluded", "IPO" : "Excluded", "Acquired" : "Excluded", "Pre-Seed" : "Included",
        "Seed" : "Included", "Series A" : "Included", "Series B" : "Included", "Series C" : "Included", "Series D+" : "Included"}
    df2["keys_company_stage_group"] = df2["keys_company_stage"].map(group_stages)
    ordinal_stages = {"Pre-Seed" : 1, "Seed" : 2, "Series A" : 3, "Series B" : 4, "Series C" : 5, "Series D+" : 6,"Other" : np.nan, "Closed" : -1, "IPO" : 7, "Acquired" : 8}
    df2["keys_company_stage_number"] = df2["keys_company_stage"].map(ordinal_stages)
    return df2[["keys_company_stage_group", "keys_company_stage","keys_company_stage_number"]]

In [ ]:
import analysis.getStages as gs
import importlib
importlib.reload(gs)
stages = create_stages(df, **stage_features)
df = pd.concat([stages, df], axis=1)
print("Stages done.")

## 3.1.3 Preliminary Screening

In [ ]:
print(df["keys_company_stage"].value_counts())
print(df["keys_company_stage"].value_counts(normalize=True))
print(df["keys_company_stage"].value_counts().sum())

Removed Other, Closed, Acquired & IPO groups

In [ ]:
df = df.loc[df['keys_company_stage_group'] != "Excluded"]

In [ ]:
print(df["keys_company_stage"].value_counts())
print(df["keys_company_stage"].value_counts(normalize=True))
print(df["keys_company_stage"].value_counts().sum())

In [ ]:
order_1 = ["Pre-Seed", "Seed", "Series A", "Series B", "Series C", "Series D+"]

f, ax = plt.subplots()
ax.set_xlim(xmax=30)
sns.boxplot(
    x="confidence_context_broader_company_age_number", 
    y="keys_company_stage",
    order=order_1,data=df, fliersize=0, ax=ax)

Removed companies older than 15 years old

In [ ]:
age_old_cutoff = df["confidence_context_broader_company_age_number"][df["keys_company_stage"] == "Series D+"].quantile(0.75)

In [ ]:
age_old_cutoff

In [ ]:
df = df.loc[df['confidence_context_broader_company_age_number'] <= age_old_cutoff]

In [ ]:
print(df["keys_company_stage"].value_counts())
print(df["keys_company_stage"].value_counts(normalize=True))
print(df["keys_company_stage"].value_counts().sum())

## 3.1.4 Descriptive Statistics

Table 3. Final test dataset counts grouped by lifecycle stage.

In [ ]:
drops = [col for col in list(df) if col.startswith(("key","from","outcome","index"))]
X = df.drop(drops, axis=1)
X = X.select_dtypes(['number'])

In [ ]:
df["keys_missing_features"] = X.isnull().sum(axis=1)
len(list(df))

In [ ]:
groups = [df["keys_company_stage"]]
columns = ["keys_company_stage", "confidence_context_broader_company_age_number", "confidence_validation_funding_raised_value_total_number", "confidence_validation_funding_rounds_number", "keys_missing_features"]
colnames = ["Obs", "Age (Years)", "Funding Raised (USD, millions)", "Funding Rounds (N)", "Available Features (N)"]
colfuncs = [{"N":len}, 
    OrderedDict([("Median", np.median),("IQR", lambda x: stats.iqr(x,nan_policy="omit"))]), 
    OrderedDict([("Median", lambda x: np.median(x) / 1e6), ("IQR", lambda x: stats.iqr(x,nan_policy="omit") / 1e6)]), 
    OrderedDict([("Median", np.median),("IQR", lambda x: stats.iqr(x,nan_policy="omit"))]), 
    OrderedDict([("Median", lambda x: len(list(df)) - np.median(x)), ("IQR", lambda x: stats.iqr(x,nan_policy="omit") )])]

namefunc = OrderedDict(zip(columns, colnames))
aggfunc = OrderedDict(zip(columns, colfuncs))

order_1 = ["Pre-Seed", "Seed", "Series A", "Series B", "Series C", "Series D+"]

tab = df[columns].groupby(groups)
tab = tab.agg(aggfunc)
tab.rename(columns=namefunc)

In [ ]:
groups = [df["keys_company_stage_group"]]
columns = ["keys_company_stage", "confidence_context_broader_company_age_number", "confidence_validation_funding_raised_value_total_number", "confidence_validation_funding_rounds_number", "keys_missing_features"]
colnames = ["Obs", "Age (Years)", "Funding Raised (USD, millions)", "Funding Rounds (N)", "Available Features (N)"]
colfuncs = [{"N":len}, 
    OrderedDict([("Median", np.median),("IQR", lambda x: stats.iqr(x,nan_policy="omit"))]), 
    OrderedDict([("Median", lambda x: np.median(x) / 1e6), ("IQR", lambda x: stats.iqr(x,nan_policy="omit") / 1e6)]), 
    OrderedDict([("Median", np.median), ("IQR", lambda x: stats.iqr(x,nan_policy="omit"))]),
    OrderedDict([("Median", lambda x: len(list(df)) - np.median(x)), ("IQR", lambda x: stats.iqr(x,nan_policy="omit") )])]

namefunc = OrderedDict(zip(columns, colnames))
aggfunc = OrderedDict(zip(columns, colfuncs))

tab = df[columns].groupby(groups)
tab = tab.agg(aggfunc)
tab.rename(columns=namefunc)

Table 3. Final test dataset counts grouped by company sector.

In [ ]:
industries = [x for x in list(df) if x.startswith("confidence_context_industry_category_group_list")]
values = [df[industry].value_counts()[1] for industry in industries]
values = sorted(values,reverse=True)
print(values)
names = ["Software", "Internet Services", "Media & Entertainment", "Commerce", "Mobile", "Sales & Marketing", "Information Technology", "Health Care", "Hardware", "Financial Services"]
sns.barplot(x = values, y = names, order = names)

[SKEW BY FEATURE (GROUPED BY WHETHER ZEROS-REMOVED)]

In [ ]:
X.apply(lambda x: stats.skew(x, nan_policy="omit")).astype(float).plot(kind="kde", label="Skew", xlim=(-20, 40))
X.replace(0, np.nan).apply(lambda x: stats.skew(x, nan_policy="omit")).astype(float).plot(kind="kde", label="Skew")

[KURTOSIS  BY FEATURE (GROUPED BY WHETHER ZEROS-REMOVED)]

In [ ]:
X.apply(lambda x: stats.kurtosis(x, nan_policy="omit")).astype(float).plot(kind="kde", label="Skew", xlim=(-200, 400))
X.replace(0, np.nan).apply(lambda x: stats.kurtosis(x, nan_policy="omit")).astype(float).plot(kind="kde", label="Skew")

[BOXPLOT OF FEATURES]

In [ ]:
X.apply(lambda x: stats.iqr(x, nan_policy="omit")).astype(float).plot(kind="kde", label="IQR", xlim=(0, 5e8))
X.replace(0, np.nan).apply(lambda x: stats.iqr(x, nan_policy="omit")).astype(float).plot(kind="kde", label="IQR", xlim=(0, 5e8))

[COVARIANCE MATRIX]

In [ ]:
#sns.heatmap(X.corr(), square=True, xticklabels =False, yticklabels=False, vmin=0.5)
#pd.DataFrame(np.where(abs(X.corr().stack()) <= 1, True, False)).stack().value_counts()
corrs = {thresh/10: pd.DataFrame(np.where(abs(X.corr().stack()) >= thresh/10, True, False)).stack().value_counts(normalize=True)[True] for thresh in range(0, 11)}
ax = pd.DataFrame.from_dict(data=corrs, orient="index").plot(kind="line")

corrs = {thresh/10: pd.DataFrame(np.where(abs(X.replace(0, np.nan).corr().stack()) >= thresh/10, True, False)).stack().value_counts(normalize=True)[True] for thresh in range(0, 11)}
pd.DataFrame.from_dict(data=corrs, orient="index").plot(kind="line",ax=ax)

# 3.2 Pre-Processing

## 3.2.1 Imputation

(!) Figure 3. Number of missing features per observation (histogram).

In [ ]:
g = sns.distplot(df["keys_missing_features"], bins=100, kde=False)
g.set(xlim = (0,None))

(!) Figure 3. Number of missing observations per feature (histogram).

In [ ]:
missing_by_col =  X.isnull().sum(axis=0)
g = sns.distplot(missing_by_col, bins=100, kde=False)
g.set(xlim = (0,None))

Figure 3. Mean, median and mode of features (grouped bar plot).

In [ ]:
means_by_col = X.mean(axis=0)#.dropna()
means_by_col.plot(kind="kde", label="Mean")

medians_by_col = X.median(axis=0)#.dropna()
medians_by_col.plot(kind="kde", label="Median")

modes_by_col = X.mode(axis=0).T[0]
modes_by_col.plot(kind="kde",label="Mode")

Figure 3. ROC Curve for different imputations - mean, median, mode (line plot).

## 3.2.2 Transformation

--> Apply Imputation

In [ ]:
from sklearn.preprocessing import Imputer

X = X.dropna(axis=1, how = "all")
imp = Imputer(strategy="median")
X_imp = imp.fit_transform(X)
X_imp = pd.DataFrame(X_imp, index=X.index, columns=list(X))

--> Shift to positive numbers only

In [ ]:
mins_by_col = X_imp.min(axis=0)#.dropna()
mins_by_col.plot(kind="kde", label="Minimum")

In [ ]:
X_pos = X_imp.subtract(X_imp.min(axis=0))
X_pos.describe()

--> Start transformations

In [ ]:
funding = X_pos["confidence_validation_funding_raised_value_total_number"]

figure = plt.figure()

ax0 = plt.subplot(151)
funding.plot(kind="hist", bins=100)
plt.title("Original - No Transformation") 
print("Skewness (Original): {0:.2f}".format(stats.skew(funding))) 

figure.add_subplot(152, sharey=ax0)
boxcox = pd.Series(stats.boxcox(funding + 1)[0], index=X_pos.index)
boxcox.plot(kind="hist", bins=100)
plt.title("BoxCox Transformation") 
print("Skewness (BoxCox): {0:.2f}".format(stats.skew(boxcox))) 

figure.add_subplot(153, sharey=ax0)
log1p = pd.Series(np.log1p(funding), index=X_pos.index)
log1p.plot(kind="hist", bins=100)
plt.title("Log1P Transformation") 
print("Skewness (Log1P): {0:.2f}".format(stats.skew(log1p)))

figure.add_subplot(154, sharey=ax0)
sqrt = pd.Series(funding**(1/2), index=X_pos.index)
sqrt.plot(kind="hist", bins=100)
plt.title("SQRT Transformation") 
print("Skewness (SQRT): {0:.2f}".format(stats.skew(sqrt)))

from sklearn.preprocessing import binarize

figure.add_subplot(155, sharey=ax0)
binary = pd.Series(np.where(funding == 0, 0, 1), index=X_pos.index)
binary.plot(kind="hist", bins=100)
plt.title("Binary Transformation") 
print("Skewness (Binary): {0:.2f}".format(stats.skew(binary)))

--> Apply transformations

In [ ]:
from sklearn.preprocessing import FunctionTransformer

ft = FunctionTransformer(np.log1p)
X_tf = ft.fit_transform(X_pos)
X_tf = pd.DataFrame(X_tf, index=X_pos.index, columns=list(X_pos))

--> Start scaling

In [ ]:
from sklearn.preprocessing import scale, robust_scale, minmax_scale

funding = X_tf["confidence_validation_funding_raised_value_total_number"]

figure = plt.figure()

ax0 = plt.subplot(151)
funding.plot(kind="hist", bins=100)
plt.title("Original - No Transformation") 
print("Median: {}, IQR: {}".format(funding.median(), stats.iqr(funding)))

figure.add_subplot(152, sharey=ax0)
funding_std = pd.Series(scale(funding), index=X_pos.index)
funding_std.plot(kind="hist", bins=100)
plt.title("Standard Scaling") 
print("Median: {}, IQR: {}".format(funding_std.median(), stats.iqr(funding_std)))


figure.add_subplot(153, sharey=ax0)
funding_robust = pd.Series(robust_scale(funding), index=X_pos.index)
funding_robust.plot(kind="hist", bins=100)
plt.title("Robust Scaling") 
print("Median: {}, IQR: {}".format(funding_robust.median(), stats.iqr(funding_robust)))


figure.add_subplot(154, sharey=ax0)
funding_mm = pd.Series(minmax_scale(funding), index=X_pos.index)
funding_mm.plot(kind="hist", bins=100)
plt.title("Min Max Scaling") 
print("Median: {}, IQR: {}".format(funding_mm.median(), stats.iqr(funding_mm)))

--> Apply scaling

In [ ]:
from sklearn.preprocessing import scale

X_std = scale(X_tf)
X_std = pd.DataFrame(X_std, index=X_tf.index, columns=list(X_tf))

## 3.2.3 Extraction

PCA Model

In [ ]:
#Principal Components Analysis (PCA)
from sklearn.decomposition import PCA

def screeplot(pca, X):
    y = pca.explained_variance_
    x = np.arange(len(y)) + 1
    plt.plot(x, y)
    plt.ylabel("Eigenvalue")
    plt.xlabel("Component Number")
    plt.xlim(1,None)
    return y

pca = PCA().fit(X_std)
eigenvalues = screeplot(pca,X_std)

Grouped

In [ ]:
from collections import defaultdict
from itertools import combinations

features = list(X_std)
f_map = {x: '_'.join(x.split("_")[:3]) for x in features}
f_group = defaultdict(list)
for f,g in f_map.items():
    f_group[g].append(f)
agg_group = {}
for g,l in f_group.items():
    combo = combinations(l, 2)
    for x,y in combo:
        corr = stats.spearmanr(X_std[x][:100], X_std[y][:100])[0]
        if corr < 0: X_std[y] *= -1
    tot = X_std[l].sum(axis=1)
    agg_group[g] = tot
X_grp = pd.DataFrame(agg_group)
grp_corr = X_grp.corr(method="spearman")
mask = np.zeros_like(grp_corr)
mask[np.triu_indices_from(mask)] = True
f_names = [x.split("_")[2] for x in list(X_grp)]
ax = sns.heatmap(grp_corr,square=True, mask=mask, xticklabels=f_names, yticklabels=f_names)
ax.invert_yaxis()

In [ ]:
pca = PCA().fit(X_grp)
eigenvalues = screeplot(pca,X_grp)

# 3.3 Classification

# 3.4 Experimentation

## 3.4.2 Robustness

## 3.4.3 Predictive Power

Table 3.5 Comparison of 2013 slice from 2016 dataset with original 2013 dataset.